### XGBoost Assignment

In this assignment students need to predict whether a person makes over
50K per year or not from classic adult dataset using XGBoost. 

**Data Set Information:**

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

**Attribute Information:**

Listing of attributes: >50K, <=50K.

Age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school,Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house serv,Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative,Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico,Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan,Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy,Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France,Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia,Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [7]:
import numpy as np
import pandas as pd

X_train = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)

X_test = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test' , skiprows = 1, header = None)

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 
              'education_num','marital_status', 'occupation','relationship', 
              'race', 'sex', 'capital_gain','capital_loss', 'hours_per_week', 
              'native_country', 'wage_class']
X_train.columns = col_labels
X_test.columns = col_labels

In [8]:
X_train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [9]:
X_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [11]:
X_train.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
dtype: int64

In [12]:
X_train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'wage_class'],
      dtype='object')

In [13]:
dict_work_class = {}
count = 0
for i in X_train.workclass.unique():
    dict_work_class[i]=count
    count+=1

dict_sex={}
count=0
for i in X_train.sex.unique():
    dict_sex[i]=count
    count+=1
    
dict_education={}
count=0
for i in X_train.education.unique():
    dict_education[i]=count
    count+=1

dict_marital_status={}
count=0
for i in X_train.marital_status.unique():
    dict_marital_status[i]=count
    count+=1

dict_occupation={}
count=0
for i in X_train.occupation.unique():
    dict_occupation[i]=count
    count+=1
    
dict_relationship={}
count=0
for i in X_train.relationship.unique():
    dict_relationship[i]=count
    count+=1
    
dict_race={}
count=0
for i  in X_train.race.unique():
    dict_race[i]=count
    count+=1
    
dict_native_country={}
count=0
for i in X_train.native_country.unique():
    dict_native_country[i]=count
    count+=1
    
dict_wage_class={}
count=0
for i in X_train.wage_class.unique():
    dict_wage_class[i]=count
    count+=1

In [14]:
X_train['sex']=X_train['sex'].map(dict_sex)
X_train['education']=X_train['education'].map(dict_education)
X_train['wage_class']=X_train['wage_class'].map(dict_wage_class)
X_train['native_country']=X_train['native_country'].map(dict_native_country)
X_train['race']=X_train['race'].map(dict_race)
X_train['occupation']=X_train['occupation'].map(dict_occupation)
X_train['marital_status']=X_train['marital_status'].map(dict_marital_status)
X_train['workclass']=X_train['workclass'].map(dict_work_class)
X_train['relationship']=X_train['relationship'].map(dict_relationship)

In [15]:
X_train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [16]:
X_test.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'wage_class'],
      dtype='object')

In [17]:
dict_work_class = {}
count = 0
for i in X_test.workclass.unique():
    dict_work_class[i]=count
    count+=1

dict_sex={}
count=0
for i in X_test.sex.unique():
    dict_sex[i]=count
    count+=1
    
dict_education={}
count=0
for i in X_test.education.unique():
    dict_education[i]=count
    count+=1

dict_marital_status={}
count=0
for i in X_test.marital_status.unique():
    dict_marital_status[i]=count
    count+=1

dict_occupation={}
count=0
for i in X_test.occupation.unique():
    dict_occupation[i]=count
    count+=1
    
dict_relationship={}
count=0
for i in X_test.relationship.unique():
    dict_relationship[i]=count
    count+=1
    
dict_race={}
count=0
for i  in X_test.race.unique():
    dict_race[i]=count
    count+=1
    
dict_native_country={}
count=0
for i in X_test.native_country.unique():
    dict_native_country[i]=count
    count+=1
    
dict_wage_class={}
count=0
for i in X_test.wage_class.unique():
    dict_wage_class[i]=count
    count+=1

In [18]:
X_test['sex']=X_test['sex'].map(dict_sex)
X_test['education']=X_test['education'].map(dict_education)
X_test['wage_class']=X_test['wage_class'].map(dict_wage_class)
X_test['native_country']=X_test['native_country'].map(dict_native_country)
X_test['race']=X_test['race'].map(dict_race)
X_test['occupation']=X_test['occupation'].map(dict_occupation)
X_test['marital_status']=X_test['marital_status'].map(dict_marital_status)
X_test['workclass']=X_test['workclass'].map(dict_work_class)
X_test['relationship']=X_test['relationship'].map(dict_relationship)

In [19]:
X_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,0,226802,0,7,0,0,0,0,0,0,0,40,0,0
1,38,0,89814,1,9,1,1,1,1,0,0,0,50,0,0
2,28,1,336951,2,12,1,2,1,1,0,0,0,40,0,1
3,44,0,160323,3,10,1,0,1,0,0,7688,0,40,0,1
4,18,2,103497,3,10,0,3,0,1,1,0,0,30,0,0


In [20]:
X=X_train.drop("wage_class", axis=1)
Y=X_train["wage_class"]


In [21]:
test_x=X_test.drop("wage_class",axis=1)
test_y=X_test["wage_class"]

In [23]:
X=X.values
y=Y.values
X_test=test_x.values
y_test=test_y.values

In [40]:
from xgboost.sklearn import XGBClassifier
params={
    'objective':'binary:logistic',
    'max_depth':5,
    'learning_rate': 1.0,
    'silent':1,
    'n_estimators':5
}
model=XGBClassifier(**params).fit(X,y)
y_preds=model.predict(X_test)
y_preds

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test,y_preds)
acc

0.8485965235550642

We have achieved accuracy of 84%